In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('realest.csv')
df.head()

,Price,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
0,53.0,2.0,967.0,5.0,39.0,652.0,1.5,0.0,0.0
1,55.0,2.0,815.0,5.0,33.0,1000.0,1.0,2.0,1.0
2,56.0,3.0,900.0,5.0,35.0,897.0,1.5,1.0,0.0
3,58.0,3.0,1007.0,6.0,24.0,964.0,1.5,2.0,0.0
4,64.0,3.0,1100.0,7.0,50.0,1099.0,1.5,1.5,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      156 non-null    float64
 1   Bedroom    156 non-null    float64
 2   Space      146 non-null    float64
 3   Room       156 non-null    float64
 4   Lot        146 non-null    float64
 5   Tax        147 non-null    float64
 6   Bathroom   156 non-null    float64
 7   Garage     156 non-null    float64
 8   Condition  156 non-null    float64
dtypes: float64(9)
memory usage: 11.2 KB


In [4]:
df.describe()

,Price,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
count,156.000000,156.000000,146.000000,156.000000,146.000000,147.000000,156.000000,156.000000,156.000000
mean,56.474359,3.166667,1097.246575,6.500000,32.808219,911.707483,1.480769,0.846154,0.230769
std,12.875307,1.348037,462.540698,1.675247,8.457859,443.263430,0.529408,0.808454,0.422682
min,32.000000,1.000000,539.000000,4.000000,24.000000,418.000000,1.000000,0.000000,0.000000
25%,46.000000,2.000000,805.250000,5.000000,25.000000,652.500000,1.000000,0.000000,0.000000
50%,55.000000,3.000000,965.500000,6.000000,30.000000,821.000000,1.500000,1.000000,0.000000
75%,65.000000,4.000000,1220.500000,7.000000,37.000000,1012.500000,2.000000,1.500000,0.000000
max,90.000000,8.000000,2295.000000,12.000000,50.000000,2752.000000,3.000000,2.000000,1.000000


In [5]:
df.isna().sum()

Price         1
Bedroom       1
Space        11
Room          1
Lot          11
Tax          10
Bathroom      1
Garage        1
Condition     1
dtype: int64

In [6]:
#dropping single value using dropna
df_new = df.copy()
cols = ['Price','Bedroom','Room','Bathroom','Garage','Condition']
df_new = df_new.dropna(subset=cols)
df_new.isna().sum()

Price         0
Bedroom       0
Space        10
Room          0
Lot          10
Tax           9
Bathroom      0
Garage        0
Condition     0
dtype: int64

In [7]:
col = ['Space','Lot','Tax']
df_final = df_new.copy()
df_final[col] = df_final[col].fillna(-999)


In [8]:
df_final.isna().sum()

Price        0
Bedroom      0
Space        0
Room         0
Lot          0
Tax          0
Bathroom     0
Garage       0
Condition    0
dtype: int64

In [9]:
#finding covariance with Price column to understand the realtionship with other features in dataframe

price_corr = df_final.corr()['Price']
price_corr_sorted = price_corr.sort_values(ascending=False)
price_corr_sorted

Price        1.000000
Room         0.578480
Garage       0.553105
Bathroom     0.538485
Space        0.443076
Tax          0.395465
Bedroom      0.302451
Condition    0.079336
Lot          0.062433
Name: Price, dtype: float64

In [10]:
## We found out that Room,Garage,Bathroom & Space are the most correlated features to the traget variable as Prices of chicago houses.

In [11]:
## Train - test split & fiting linear regression model 
features = ['Room','Garage','Bathroom','Space','Tax','Bedroom','Condition','Lot']
X = df_final.loc[:,features]
y = df_final.loc[:,['Price']]


In [12]:
df_final_shuffle = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
split_ratio = 0.8
split_index = int(len(df_final_shuffle) * split_ratio)

X_train = X.iloc[:split_index,:]
X_test = X.iloc[:split_index,:]
y_train = y.iloc[:split_index,:]
y_test = y.iloc[:split_index,:]

# Display the shapes of the datasets
print("Training set X:", X_train.shape, "y:", y_train.shape)
print("Testing set X:", X_test.shape, "y:", y_test.shape)

Training set X: (124, 8) y: (124, 1)
Testing set X: (124, 8) y: (124, 1)


In [13]:
# fiting the model

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

residuals = y_test.values.flatten() - y_pred.flatten()

mse = np.mean(residuals ** 2)



In [14]:
# Print the Mean Squared Error
print(f"Mean Squared Error (manual calculation): {mse:.2f}")

# Print predictions and actual values in a formatted way
print("\nPredictions vs Actual Values:")
print(f"{'Predicted Price':<20} {'Actual Price':<20} {'Difference':<20}")
print("-" * 60)

# Print each predicted value alongside the actual value and their difference
for predicted, actual in zip(y_pred.flatten(), y_test.values.flatten()):
    difference = predicted - actual
    print(f"{predicted:<20.2f} {actual:<20.2f} {difference:<20.2f}")

Mean Squared Error (manual calculation): 55.34

Predictions vs Actual Values:
Predicted Price      Actual Price         Difference          
------------------------------------------------------------
49.81                53.00                -3.19               
58.21                55.00                3.21                
51.94                56.00                -4.06               
62.13                58.00                4.13                
65.00                64.00                1.00                
60.27                44.00                16.27               
52.19                49.00                3.19                
71.36                70.00                1.36                
62.89                72.00                -9.11               
75.49                82.00                -6.51               
81.78                85.00                -3.22               
46.91                45.00                1.91                
46.18                47.00                

In [15]:

# Define the number of folds
K = 5

# Shuffle the dataset
shuffled_indices = np.random.permutation(len(X))
X_shuffled = X.iloc[shuffled_indices].reset_index(drop=True)
y_shuffled = y.iloc[shuffled_indices].reset_index(drop=True)

# Split the data into K folds
fold_size = len(X) // K
mse_list = []

for fold in range(K):
    # Determine the indices for the validation fold
    val_start = fold * fold_size
    val_end = val_start + fold_size
    
    # Create the training and validation sets
    X_train = pd.concat([X_shuffled.iloc[:val_start], X_shuffled.iloc[val_end:]])
    y_train = pd.concat([y_shuffled.iloc[:val_start], y_shuffled.iloc[val_end:]])
    
    X_val = X_shuffled.iloc[val_start:val_end]
    y_val = y_shuffled.iloc[val_start:val_end]
    
    # Fit the linear regression model (manually)
    # Using the normal equation: (X^T * X)^(-1) * X^T * y
    X_train_matrix = X_train.values
    y_train_matrix = y_train.values.flatten()
    
    # Adding a bias (intercept) term to X
    X_train_biased = np.c_[np.ones(X_train_matrix.shape[0]), X_train_matrix]
    
    # Calculate coefficients using the normal equation
    coeffs = np.linalg.inv(X_train_biased.T @ X_train_biased) @ (X_train_biased.T @ y_train_matrix)
    
    # Make predictions on the validation set
    X_val_matrix = X_val.values
    X_val_biased = np.c_[np.ones(X_val_matrix.shape[0]), X_val_matrix]  # Add bias term
    
    y_preds = X_val_biased @ coeffs
    
    # Calculate MSE manually
    mse_fold = np.mean((y_val.values.flatten() - y_preds) ** 2)
    mse_list.append(mse_fold)

# Calculate the average MSE across all folds
average_mse = np.mean(mse_list)
print(f"Optimized Mean Squared Error (MSE) from K-Fold Cross-Validation: {average_mse:.2f}")


Optimized Mean Squared Error (MSE) from K-Fold Cross-Validation: 70.96


In [16]:
# Print predicted values
print("Predicted values:\n", y_pred)

Predicted values:
 [[49.80903423]
 [58.20622641]
 [51.94182831]
 [62.12713355]
 [64.99893692]
 [60.26746273]
 [52.19004803]
 [71.36484195]
 [62.88662409]
 [75.48831791]
 [81.78040037]
 [46.90973757]
 [46.17643643]
 [50.55824907]
 [59.08365289]
 [47.08123429]
 [58.45007837]
 [59.62252142]
 [55.47114672]
 [50.12006328]
 [45.28669209]
 [32.1971244 ]
 [46.63777343]
 [52.00712679]
 [59.84297834]
 [64.43953012]
 [53.84821355]
 [62.52998102]
 [56.29744073]
 [62.21482916]
 [65.22104645]
 [64.32538096]
 [56.27078519]
 [71.4464095 ]
 [59.57533363]
 [71.62563449]
 [81.77061969]
 [42.71651844]
 [46.00729423]
 [54.93489036]
 [54.65609234]
 [51.21236659]
 [62.55298939]
 [68.37943062]
 [55.61996154]
 [45.81079022]
 [41.32874169]
 [45.92732105]
 [42.14316719]
 [48.36210641]
 [59.74166394]
 [64.3220893 ]
 [45.5823813 ]
 [53.84770319]
 [56.18676491]
 [62.32248931]
 [57.56275622]
 [60.25723856]
 [57.43077523]
 [75.35977028]
 [68.20239451]
 [80.06603985]
 [86.30871088]
 [42.33655523]
 [42.28119371]
 [46.2